In [7]:
import pandas as pd
import os
import fnmatch

In [4]:
df_us_2020 =pd.read_csv('../data/RealData/2022_US_cohort.csv')

In [5]:
df_us_2020.head()

,birth_year,fridge_freezer,washing_machine,tumble_dryer,dishwasher,microwave,burglaries,car_crime,drunks,muggings,...,universal_credit,hosp_visits,gp_visits,depression,birth_month,academic_year,time,child_ages,age,weight
0,1969.0,1.0,1.0,1.0,1.0,1.0,-9.0,-9.0,-9.0,-9.0,...,0.0,2.0,1.0,-8.0,11.0,1969.0,2022,childless,52,-9.0
1,1970.0,1.0,1.0,1.0,1.0,1.0,-9.0,-9.0,-9.0,-9.0,...,-7.0,-7.0,-7.0,-7.0,1.0,1969.0,2022,childless,52,-9.0
2,1957.0,1.0,1.0,0.0,1.0,1.0,-9.0,-9.0,-9.0,-9.0,...,0.0,1.0,1.0,-8.0,8.0,1956.0,2022,childless,64,-9.0
3,1986.0,1.0,1.0,1.0,1.0,1.0,-9.0,-9.0,-9.0,-9.0,...,0.0,2.0,1.0,-8.0,6.0,1985.0,2022,3_6,36,-9.0
4,1977.0,1.0,1.0,1.0,1.0,1.0,-9.0,-9.0,-9.0,-9.0,...,0.0,2.0,2.0,-8.0,2.0,1976.0,2022,3_6,44,-9.0


In [27]:
for item in sorted(df_us_2020.columns):
    print(item)


S7_mental_health
S7_physical_health
SF_12
SF_12p
academic_year
age
alcohol_spending
behind_on_bills
birth_month
birth_year
burglaries
car_crime
child_ages
clinical_depression
council_tax
depression
dishwasher
drunks
education_state
emp_type
energy_in_rent
ethnicity
financial_situation
fridge_freezer
fruit_days
fruit_per_day
future_financial_situation
gas_electric_combined
ghq_depression
ghq_happiness
gp_visits
gross_pay_se
gross_paypm
has_electric
has_gas
has_none
has_oil
has_other
health_limits_social
heating
hh_composition
hh_int_m
hh_int_y
hh_mortgage
hh_netinc
hh_rent
hhsize
hidp
hosp_visits
hourly_rate
housing_tenure
jb_inc_per
job_duration_m
job_duration_y
job_hours
job_hours_se
job_inc
job_industry
job_occupation
job_sec
job_sector
labour_state_raw
likely_move
loneliness
marstat
microwave
muggings
ncigs
ndrinks
newest_education_state
nkids
nkids_ind_new
nkids_ind_raw
oecd_equiv
pain_interfere_work
phealth_limits_modact
phealth_limits_stairs
phealth_limits_work_type
pidp
racial_a

In [5]:
df_economic = pd.read_csv('../data/RealData/2011_census_economic.csv')

In [6]:
df_economic.head()

,areacode,self_employed,in_paid_employment,unemployed,retired,other,looking_after_home,student,long_term_sick_disabled,total
0,E01000001,243,694,39,263,18,36,45,12,1350
1,E01000002,250,669,16,257,18,29,48,6,1293
2,E01000003,191,578,58,247,18,17,89,44,1242
3,E01000005,55,426,54,115,25,44,53,49,821
4,E01000006,168,613,93,125,38,105,118,49,1309


In [20]:


def read_csv_files(file_list):
    # List to store DataFrames
    dataframes = []
    totals_dataframes = []

    # Read each CSV file into a DataFrame
    for file in file_list:
        df = pd.read_csv(file)
        # Exclude the 'totals' column from the main DataFrame
        df_main = df.drop(columns=['total'])
        dataframes.append(df_main)
        # Store the 'totals' column separately
        totals_dataframes.append(df[['areacode', 'total']])

    return dataframes, totals_dataframes

def merge_dataframes(dataframes, key_column='areacode'):
    # Initialize the merged DataFrame with the first DataFrame
    merged_df = dataframes[0].set_index(key_column)

    # Iterate through the remaining DataFrames
    for i, df in enumerate(dataframes[1:], start=1):
        # Rename columns to avoid duplicates
        df_renamed = df.rename(columns=lambda x: f"{x}_{i}" if x != key_column else x)
        # Merge with the current merged DataFrame
        merged_df = merged_df.join(df_renamed.set_index(key_column), how='outer')

    # Reset the index to get the key_column back as a column
    merged_df.reset_index(inplace=True)

    return merged_df

def find_missing_areacodes(dataframes, key_column='areacode'):
    # Get the set of all areacodes from the first DataFrame
    all_areacodes = set(dataframes[0][key_column])

    # Update the set with areacodes from all other DataFrames
    for df in dataframes[1:]:
        all_areacodes.update(df[key_column])

    # Find areacodes that are missing in any of the DataFrames
    missing_areacodes = set()
    for df in dataframes:
        missing_areacodes.update(all_areacodes - set(df[key_column]))

    return missing_areacodes

def check_totals_consistency(totals_dataframes):
    # Combine all totals DataFrames into one
    combined_totals = pd.concat(totals_dataframes, ignore_index=True)

    # Pivot the combined totals to check consistency
    pivot_totals = combined_totals.pivot(index='areacode', columns='file', values='total')

    # Check for inconsistencies
    inconsistencies = pivot_totals.apply(lambda x: x.nunique() > 1, axis=1)

    # Get the areacodes with inconsistent totals
    inconsistent_areacodes = pivot_totals[inconsistencies].index.tolist()

    return inconsistent_areacodes



# Read all CSV files into DataFrames
dataframes, totals_dataframes = read_csv_files(file_list)

# Add a 'file' column to each totals DataFrame to track the source file
for i, df in enumerate(totals_dataframes):
    df['file'] = f'file{i+1}'

# Merge the DataFrames on the 'areacode' column
merged_df = merge_dataframes(dataframes)

# Find missing areacodes
missing_areacodes = find_missing_areacodes(dataframes)

# Check for inconsistencies in the 'totals' column
inconsistent_areacodes = check_totals_consistency(totals_dataframes)

# Print the merged DataFrame
print("Merged DataFrame:")
print(merged_df)

# Print the missing areacodes
print("\nMissing areacodes:")
print(missing_areacodes)

# Print the inconsistent areacodes
# print("\nInconsistent areacodes (different total):")
# print(len(inconsistent_areacodes))
# print(inconsistent_areacodes)


Merged DataFrame:
        areacode  level_1_to_2  level_3  level_4_and_above  none  \
0      E01000001           110      168               1047    25   
1      E01000002           112      127               1024    30   
2      E01000003           192      183                706   161   
3      E01000005           159      179                283   200   
4      E01000006           324      316                431   238   
...          ...           ...      ...                ...   ...   
41724  W01001954           265      172                708   128   
41725  W01001955           348      303                332   532   
41726  W01001956           511     1313                590   241   
41727  W01001957           229      219                202   284   
41728  W01001958           215      314                505   187   

       very_good_health_1  good_health_1  fair_health_1  poor_health_1  \
0                     743            452            126             27   
1                

In [33]:

for item in sorted(merged_df.columns):
    print(item, end=" ")
    print(merged_df[item].unique())
print(len(merged_df.columns))

areacode ['E01000001' 'E01000002' 'E01000003' ... 'W01001956' 'W01001957'
 'W01001958']
asian_4 [ 121   86  155 ... 1378 1441 1403]
black_4 [  8   4  35  65 135 278 360 168 380 113 322 180 159 295 175 205 140 223
 207 158 132 184 157 139 219 147 220 142 141 203 193 126  70 104 177 215
 213 109 276 252 170 433 373 181 444 261 258 201 164 202 247 264 309 122
 162  92  75 108 190 249 153 150 218 192 171 173 212 189 160 361 287 284
 286 548 310 378 316 436 133 321 271 298 341 176 194 208 246 209 312  30
  36  38  71  43  29  90  56  59 134 350 371 186 169 237 136 179 210  54
  28 174  74  79 216 156  45 229 188 370 260 167  85  69  95  81 125 127
  93  66  62  24  47  64 118  46  49 120  58  77  25 196  19  72 292  39
  78 233  86  42  53  44  50  41  20  12  34  26  17  27  57  96  55  63
 187 119 110  67 128  60 102 165  21  16  15 101 111 151 106 149  99 114
  31  32 105  37  84  13  91 206 143  51  48  52 146  87 116 117 182  88
 289 154  22  11  18  33  98 243  23  61  97 423 228 323 

In [31]:
for item in sorted(df_us_2020.columns):
    print(item, end=" ")
    print(df_us_2020[item].unique())


S7_mental_health [ 3. -7.  4.  5.  2. -2. -1.  1. -9. -8.]
S7_physical_health [ 3. -7.  5.  4.  2.  1. -2. -1. -9. -8.]
SF_12 [39.73 -7.   49.99 ... 71.14 12.22 10.99]
SF_12p [37.29 -7.   36.78 ... 23.89 16.74 72.18]
academic_year [ 1.969e+03  1.956e+03  1.985e+03  1.976e+03  1.962e+03  1.959e+03
  1.990e+03  1.983e+03  1.972e+03  1.946e+03  1.979e+03  1.978e+03
  2.002e+03  1.999e+03  1.973e+03  1.968e+03  1.945e+03  1.960e+03
  1.944e+03  1.942e+03  1.967e+03  1.998e+03  1.994e+03  1.963e+03
  1.961e+03  1.948e+03  1.938e+03  2.005e+03  1.966e+03  1.970e+03
  1.965e+03  1.986e+03  1.991e+03  1.957e+03  1.952e+03  1.947e+03
  1.953e+03  1.949e+03  1.980e+03  1.984e+03  1.982e+03  1.951e+03
  1.981e+03  1.992e+03  1.989e+03  1.987e+03  1.975e+03  1.974e+03
  1.988e+03  1.964e+03  2.001e+03  1.977e+03  1.958e+03  1.955e+03
  1.971e+03  1.941e+03  1.997e+03  1.993e+03  1.996e+03  1.943e+03
  1.935e+03  2.004e+03  2.000e+03  2.003e+03  1.995e+03  1.933e+03
  1.921e+03  1.950e+03  1.940e+0